In [175]:
import pandas as pd
import numpy as np
import datetime
import os

from DataTool import DataLoader
data_loader = DataLoader()
analysis_folder_name = 'komek_test1'
baseline_csv_name = 'credit-0418.csv'

w_mean   = 0.7
w_std    = 0.3

w_recall = 0.3
w_auc    = 0.7
w_gmean  = 0

best_result_pick_num = 10
pd.set_option('display.float_format',lambda x : '%.5f' % x)
print(datetime.datetime.now())

2022-04-20 13:15:03.702512


In [176]:
baseline = data_loader.load_csv_to_pandas(f'./result/baseline/{baseline_csv_name}')
results  = data_loader.load_csv_to_pandas(f'./result/{analysis_folder_name}/result.csv')
baseline

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std
0,MWMOTE,0.38863,0.18142,0.64400,0.04640,0.56468,0.08697,0.36461,0.13976,0.32243,0.06822
1,adasyn,0.38765,0.19189,0.64296,0.04657,0.55954,0.08763,0.36767,0.13272,0.32364,0.06884
2,basic_smote,0.32320,0.13680,0.63856,0.05004,0.54276,0.09872,0.40628,0.10791,0.32804,0.05004
3,bordered_smote,0.34329,0.11999,0.64608,0.04334,0.56154,0.08448,0.38502,0.10610,0.33691,0.04537
4,instance_hardness_threshold,0.75989,0.12777,0.74888,0.02317,0.74157,0.03060,0.18859,0.04159,0.29490,0.03360
5,near_miss,0.74466,0.14576,0.62662,0.03935,0.59550,0.05739,0.11418,0.04220,0.18895,0.04174
6,no_resampling,0.14555,0.08909,0.56635,0.03882,0.35100,0.14578,0.52195,0.09582,0.20632,0.11122
7,one_sided_selection,0.15279,0.08997,0.56947,0.03926,0.36135,0.14431,0.51586,0.09422,0.21380,0.11159
8,random_over_sampling,0.44826,0.25400,0.67490,0.08848,0.59883,0.17286,0.31272,0.10135,0.30565,0.07318
9,random_under_sampling,0.65943,0.18448,0.73414,0.06312,0.71295,0.12426,0.23067,0.08254,0.30777,0.05180


In [177]:
#! drop some resampler
# baseline.drop(3, axis=0,inplace=True)
# baseline
# baseline.index

In [178]:
print(results.shape)
# drop columns
drop_columns = ['resample_time', 'train_time', 'support', 'shuffle', 'original_maj_min', 'train_args']
results = results.drop(drop_columns, axis=1)
# results.head(5)

(104, 17)


In [179]:
#! results先根据id排序
sorted_results = results.sort_values(by='id')
# sorted_results.head(15)

In [180]:
# group by args
grouped_results = sorted_results.groupby(['resampler','resample_args']) \
                                .agg(recall_mean=('recall', 'mean'),
                                     recall_std=('recall', 'std'),
                                     auc_mean=('auc', 'mean'),
                                     auc_std=('auc', 'std'),
                                     gmean_mean=('gmean', 'mean'),
                                     gmean_std=('gmean', 'std'),
                                     precision_mean=('precision', 'mean'),
                                     precision_std=('precision', 'std'),
                                     fscore_mean=('fscore', 'mean'),
                                     fscore_std=('fscore', 'std'))
# grouped_results.head(5)

In [181]:
#! add rank info
# grouped_results['recall_mean_rank']  =  grouped_results['recall_mean'].rank(method='dense', ascending=False)
# grouped_results['recall_std_rank']   =  grouped_results['recall_std'].rank(method='dense', ascending=False)
# grouped_results['auc_mean_rank']     =  grouped_results['auc_mean'].rank(method='dense', ascending=False)
# grouped_results['auc_std_rank']      =  grouped_results['auc_std'].rank(method='dense', ascending=False)
# grouped_results['gmean_mean_rank']   =  grouped_results['gmean_mean'].rank(method='dense', ascending=False)
# grouped_results['gmean_std_rank']    =  grouped_results['gmean_std'].rank(method='dense', ascending=False)
grouped_results['recall_mean_norm']  =  (grouped_results['recall_mean'] - grouped_results['recall_mean'].min()) / (grouped_results['recall_mean'].max() - grouped_results['recall_mean'].min())
grouped_results['recall_std_norm']   =  (grouped_results['recall_std']  - grouped_results['recall_std'].max())  / -(grouped_results['recall_std'].max()  - grouped_results['recall_std'].min())
grouped_results['auc_mean_norm']     =  (grouped_results['auc_mean']    - grouped_results['auc_mean'].min())    / (grouped_results['auc_mean'].max()    - grouped_results['auc_mean'].min())
grouped_results['auc_std_norm']      =  (grouped_results['auc_std']     - grouped_results['auc_std'].max())     / -(grouped_results['auc_std'].max()     - grouped_results['auc_std'].min())
grouped_results['gmean_mean_norm']   =  (grouped_results['gmean_mean']  - grouped_results['gmean_mean'].min())  / (grouped_results['gmean_mean'].max()  - grouped_results['gmean_mean'].min())
grouped_results['gmean_std_norm']    =  (grouped_results['gmean_std']   - grouped_results['gmean_std'].max())   / -(grouped_results['gmean_std'].max()   - grouped_results['gmean_std'].min())
grouped_results['score']             =  w_recall * ( w_mean * grouped_results['recall_mean_norm'] + w_std * grouped_results['recall_std_norm']) + \
                                        w_auc    * ( w_mean * grouped_results['auc_mean_norm']    + w_std * grouped_results['auc_std_norm'   ]) + \
                                        w_gmean  * ( w_mean * grouped_results['gmean_mean_norm']  + w_std * grouped_results['gmean_std_norm' ])
grouped_results = grouped_results.sort_values(by='score', ascending=False)
grouped_results.head(best_result_pick_num)

recall_mean  recall_std  \
resampler             resample_args                                             
komek_iht             {'komek_k': 1}                      0.74721     0.14909   
                      {'komek_k': 16}                     0.84567     0.11554   
komek_rus             {'komek_k': 16}                     0.66914     0.18064   
                      {'komek_k': 1}                      0.64442     0.18423   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}       0.43708     0.10550   
                      {'komek_k': 16, 'komek_n': 10}      0.43562     0.11174   
                      {'komek_k': 16, 'komek_n': 6}       0.43562     0.11174   

                                                      auc_mean  auc_std  \
resampler             resample_args                                       
komek_iht             {'komek_k': 1}                   0.74684  0.02805   
                      {'komek_k': 16}                  0.72114  0.01110   
komek_rus             {'komek_k': 16}                  0.72989  0.05747   
                      {'komek_k': 1}                   0.72802  0.06201   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}    0.68342  0.04233   
                      {'komek_k': 16, 'komek_n': 10}   0.68191  0.04523   
                      {'komek_k': 16, 'komek_n': 6}    0.68191  0.04523   

                                                      gmean_mean  gmean_std  \
resampler             resample_args                                           
komek_iht             {'komek_k': 1}                     0.73698    0.04219   
                      {'komek_k': 16}                    0.70124    0.01769   
komek_rus             {'komek_k': 16}                    0.71048    0.10999   
                      {'komek_k': 1}                     0.70484    0.12269   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}      0.62958    0.09082   
                      {'komek_k': 16, 'komek_n': 10}     0.62695    0.09762   
                      {'komek_k': 16, 'komek_n': 6}      0.62695    0.09762   

                                                      precision_mean  \
resampler             resample_args                                    
komek_iht             {'komek_k': 1}                         0.19917   
                      {'komek_k': 16}                        0.14605   
komek_rus             {'komek_k': 16}                        0.22339   
                      {'komek_k': 1}                         0.22665   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}          0.34793   
                      {'komek_k': 16, 'komek_n': 10}         0.33892   
                      {'komek_k': 16, 'komek_n': 6}          0.33892   

                                                      precision_std  \
resampler             resample_args                                   
komek_iht             {'komek_k': 1}                        0.06062   
                      {'komek_k': 16}                       0.04400   
komek_rus             {'komek_k': 16}                       0.09115   
                      {'komek_k': 1}                        0.06409   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}         0.08210   
                      {'komek_k': 16, 'komek_n': 10}        0.07302   
                      {'komek_k': 16, 'komek_n': 6}         0.07302   

                                                      fscore_mean  fscore_std  \
resampler             resample_args                                             
komek_iht             {'komek_k': 1}                      0.30074     0.04006   
                      {'komek_k': 16}                     0.24281     0.04432   
komek_rus             {'komek_k': 16}                     0.29906     0.04184   
                      {'komek_k': 1}                      0.30601     0.05414   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}       0.36693     0.06823   
                      {'komek_k': 16, 'komek_n': 10}      0.36191     0.07340   

In [182]:
# slice_col  = ['recall_mean', 'recall_std', 'auc_mean', 'auc_std', 'gmean_mean', 'gmean_std']
best_result = grouped_results.iloc[:best_result_pick_num, 0:10]
best_result['resampler'] = grouped_results.index[:best_result_pick_num]
best_result

recall_mean  recall_std  \
resampler             resample_args                                             
komek_iht             {'komek_k': 1}                      0.74721     0.14909   
                      {'komek_k': 16}                     0.84567     0.11554   
komek_rus             {'komek_k': 16}                     0.66914     0.18064   
                      {'komek_k': 1}                      0.64442     0.18423   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}       0.43708     0.10550   
                      {'komek_k': 16, 'komek_n': 10}      0.43562     0.11174   
                      {'komek_k': 16, 'komek_n': 6}       0.43562     0.11174   

                                                      auc_mean  auc_std  \
resampler             resample_args                                       
komek_iht             {'komek_k': 1}                   0.74684  0.02805   
                      {'komek_k': 16}                  0.72114  0.01110   
komek_rus             {'komek_k': 16}                  0.72989  0.05747   
                      {'komek_k': 1}                   0.72802  0.06201   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}    0.68342  0.04233   
                      {'komek_k': 16, 'komek_n': 10}   0.68191  0.04523   
                      {'komek_k': 16, 'komek_n': 6}    0.68191  0.04523   

                                                      gmean_mean  gmean_std  \
resampler             resample_args                                           
komek_iht             {'komek_k': 1}                     0.73698    0.04219   
                      {'komek_k': 16}                    0.70124    0.01769   
komek_rus             {'komek_k': 16}                    0.71048    0.10999   
                      {'komek_k': 1}                     0.70484    0.12269   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}      0.62958    0.09082   
                      {'komek_k': 16, 'komek_n': 10}     0.62695    0.09762   
                      {'komek_k': 16, 'komek_n': 6}      0.62695    0.09762   

                                                      precision_mean  \
resampler             resample_args                                    
komek_iht             {'komek_k': 1}                         0.19917   
                      {'komek_k': 16}                        0.14605   
komek_rus             {'komek_k': 16}                        0.22339   
                      {'komek_k': 1}                         0.22665   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}          0.34793   
                      {'komek_k': 16, 'komek_n': 10}         0.33892   
                      {'komek_k': 16, 'komek_n': 6}          0.33892   

                                                      precision_std  \
resampler             resample_args                                   
komek_iht             {'komek_k': 1}                        0.06062   
                      {'komek_k': 16}                       0.04400   
komek_rus             {'komek_k': 16}                       0.09115   
                      {'komek_k': 1}                        0.06409   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}         0.08210   
                      {'komek_k': 16, 'komek_n': 10}        0.07302   
                      {'komek_k': 16, 'komek_n': 6}         0.07302   

                                                      fscore_mean  fscore_std  \
resampler             resample_args                                             
komek_iht             {'komek_k': 1}                      0.30074     0.04006   
                      {'komek_k': 16}                     0.24281     0.04432   
komek_rus             {'komek_k': 16}                     0.29906     0.04184   
                      {'komek_k': 1}                      0.30601     0.05414   
my_instance_threshold {'komek_k': 16, 'komek_n': 2}       0.36693     0.06823   
                      {'komek_k': 16, 'komek_n': 10}      0.36191     0.07340   

In [183]:
baseline = pd.concat([baseline, best_result], axis=0)
# baseline

In [184]:
baseline['recall_mean_norm']  =  (baseline['recall_mean'] - baseline['recall_mean'].min()) / (baseline['recall_mean'].max() - baseline['recall_mean'].min())
baseline['recall_std_norm']   =  (baseline['recall_std']  - baseline['recall_std'].max())  / -(baseline['recall_std'].max()  - baseline['recall_std'].min())
baseline['auc_mean_norm']     =  (baseline['auc_mean']    - baseline['auc_mean'].min())    / (baseline['auc_mean'].max()    - baseline['auc_mean'].min())
baseline['auc_std_norm']      =  (baseline['auc_std']     - baseline['auc_std'].max())     / -(baseline['auc_std'].max()     - baseline['auc_std'].min())
baseline['gmean_mean_norm']   =  (baseline['gmean_mean']  - baseline['gmean_mean'].min())  / (baseline['gmean_mean'].max()  - baseline['gmean_mean'].min())
baseline['gmean_std_norm']    =  (baseline['gmean_std']   - baseline['gmean_std'].max())   / -(baseline['gmean_std'].max()   - baseline['gmean_std'].min())
baseline['score']             =  w_recall * ( w_mean * baseline['recall_mean_norm']  + w_std * baseline['recall_std_norm']) + \
                                 w_auc    * ( w_mean * baseline['auc_mean_norm']     + w_std * baseline['auc_std_norm'   ]) + \
                                 w_gmean  * ( w_mean * baseline['gmean_mean_norm']   + w_std * baseline['gmean_std_norm' ])
baseline = baseline.sort_values(by='score', ascending=False)
baseline

,resampler,recall_mean,recall_std,auc_mean,auc_std,gmean_mean,gmean_std,precision_mean,precision_std,fscore_mean,fscore_std,recall_mean_norm,recall_std_norm,auc_mean_norm,auc_std_norm,gmean_mean_norm,gmean_std_norm,score
4,instance_hardness_threshold,0.75989,0.12777,0.74888,0.02317,0.74157,0.03060,0.18859,0.04159,0.29490,0.03360,0.87748,0.76544,1.00000,0.84401,1.00000,0.91681,0.92040
"(komek_iht, {'komek_k': 16})","(komek_iht, {'komek_k': 16})",0.84567,0.11554,0.72114,0.01110,0.70124,0.01769,0.14605,0.04400,0.24281,0.04432,1.00000,0.83961,0.84801,1.00000,0.89673,1.00000,0.91109
"(komek_iht, {'komek_k': 1})","(komek_iht, {'komek_k': 1})",0.74721,0.14909,0.74684,0.02805,0.73698,0.04219,0.19917,0.06062,0.30074,0.04006,0.85937,0.63621,0.98883,0.78093,0.98825,0.84215,0.88625
"(komek_rus, {'komek_k': 16})","(komek_rus, {'komek_k': 16})",0.66914,0.18064,0.72989,0.05747,0.71048,0.10999,0.22339,0.09115,0.29906,0.04184,0.74785,0.44488,0.89595,0.40068,0.92040,0.40518,0.72025
9,random_under_sampling,0.65943,0.18448,0.73414,0.06312,0.71295,0.12426,0.23067,0.08254,0.30777,0.05180,0.73399,0.42159,0.91924,0.32765,0.92673,0.31320,0.71132
"(komek_rus, {'komek_k': 1})","(komek_rus, {'komek_k': 1})",0.64442,0.18423,0.72802,0.06201,0.70484,0.12269,0.22665,0.06409,0.30601,0.05414,0.71255,0.42310,0.88571,0.34211,0.90596,0.32335,0.69356
"(my_instance_threshold, {'komek_k': 16, 'komek_n': 2})","(my_instance_threshold, {'komek_k': 16, 'komek...",0.43708,0.10550,0.68342,0.04233,0.62958,0.09082,0.34793,0.08210,0.36693,0.06823,0.41639,0.90050,0.64137,0.59634,0.71327,0.52869,0.60799
"(my_instance_threshold, {'komek_k': 16, 'komek_n': 10})","(my_instance_threshold, {'komek_k': 16, 'komek...",0.43562,0.11174,0.68191,0.04523,0.62695,0.09762,0.33892,0.07302,0.36191,0.07340,0.41431,0.86265,0.63310,0.55887,0.70653,0.48488,0.59223
"(my_instance_threshold, {'komek_k': 16, 'komek_n': 6})","(my_instance_threshold, {'komek_k': 16, 'komek...",0.43562,0.11174,0.68191,0.04523,0.62695,0.09762,0.33892,0.07302,0.36191,0.07340,0.41431,0.86265,0.63310,0.55887,0.70653,0.48488,0.59223
5,near_miss,0.74466,0.14576,0.62662,0.03935,0.59550,0.05739,0.11418,0.04220,0.18895,0.04174,0.85573,0.65638,0.33019,0.63492,0.62600,0.74420,0.53390
